<a href="https://colab.research.google.com/github/govardhan-06/Eng-Mal-Translator/blob/main/Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers[torch] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

dataset= load_dataset("Govardhan-06/flores_eng_mal")
dataset

Generating train split:   0%|          | 0/1808 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/201 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence_eng_Latn', 'sentence_mal_Mlym'],
        num_rows: 1808
    })
    test: Dataset({
        features: ['sentence_eng_Latn', 'sentence_mal_Mlym'],
        num_rows: 201
    })
})

In [ ]:
#Renaming test data to validation data
dataset['validation']=dataset.pop('test')

In [ ]:
dataset['train'][1500]

{'sentence_eng_Latn': 'Since 1988, ballot boxes must be transparent so that voters and observers can witness that no envelopes are present at the start of the vote and that no envelopes are added except those of the duly counted and authorized voters.',
 'sentence_mal_Mlym': '1988 മുതൽ ബാലറ്റ് ബോക്സുകൾ സുതാര്യമാണ്, അതിലൂടെ വോട്ടർമാർക്കും നിരീക്ഷകർക്കും വോട്ടെടുപ്പിന്റെ തുടക്കത്തിൽ ഒരു എൻ\u200cവലപ്പുകളും ഇല്ലെന്നതും ശരിയായി എണ്ണിയിട്ടുള്ളതും അംഗീകൃതവുമായ വോട്ടർമാർക്കും ഒഴികെ എൻ\u200cവലപ്പുകളൊന്നും ചേർത്തിട്ടില്ലെന്നതും കൃത്യമായി കാണാനും നിരീക്ഷിക്കാനും സാധിക്കും.'}

In [ ]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

model_ckpt="facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt,src_lang="eng_Latn", tgt_lang="mal_Mlym",return_tensors='pt')
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
dataset['train']['sentence_eng_Latn'][0]

'Many buildings are quite beautiful to look at and the view from a tall building or from a cleverly-positioned window can be a beauty to behold.'

In [ ]:
dataset['train']['sentence_mal_Mlym'][0]

'പല കെട്ടിടങ്ങളും കാണാൻ നല്ല ഭംഗിയാണ് കൂടാതെ ഉയരമുള്ള ഒരു കെട്ടിടത്തിൽ നിന്നോ അല്ലെങ്കിൽ ഉചിതമായി സ്ഥാനം നിശ്ചയിച്ച് പണിത ഒരു ജനലിലൂടെയോ കാണുന്ന കാഴ്ച്ച മനോഹരമായിരിക്കും.'

In [ ]:
eng_sent= dataset['train']['sentence_eng_Latn'][0]
mal_sent= dataset['train']['sentence_mal_Mlym'][0]

model_input = tokenizer(eng_sent, text_target=mal_sent)
model_input

{'input_ids': [256047, 60528, 237826, 2442, 79655, 56417, 202, 15166, 230, 540, 349, 56749, 5057, 9, 30530, 76412, 618, 5057, 9, 19201, 406, 924, 248105, 106074, 76, 111719, 2125, 280, 9, 93675, 202, 10071, 2736, 248075, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [256115, 42044, 86113, 34286, 248538, 44124, 103918, 41855, 12376, 30718, 114258, 144673, 102634, 248505, 53837, 6000, 86113, 34286, 248538, 12279, 63266, 248878, 59991, 5930, 248786, 26542, 9602, 36586, 88443, 172967, 1948, 17638, 164344, 26542, 6000, 41445, 248628, 134792, 17327, 214893, 35223, 34126, 3351, 237592, 248505, 238706, 248075, 2]}

In [ ]:
dataset['train'].column_names

['sentence_eng_Latn', 'sentence_mal_Mlym']

In [ ]:
max_length=128

def tokenize_function(examples):
  input= examples["sentence_eng_Latn"]
  target= examples["sentence_mal_Mlym"]
  model_input = tokenizer(input, text_target=target,max_length=max_length, truncation=True)
  return model_input

In [ ]:
tokenized_inputs=dataset.map(tokenize_function,batched=True,remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1808 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

### Fine-tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_inputs["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
batch['labels']

tensor([[256115, 190163,  25843,  98080, 113950, 196503,  13902,  10418, 248563,
         249069,  22991,  24757,  13230,   1436, 248303,   3065,  19206, 248376,
          35798, 248376,  10193,   2812,    997,  18870,  69755, 191199,  34646,
          90031,  96730, 248075,      2],
        [256115,   1214,  10390, 249171,   9152,   8418, 248400,  10418, 248801,
          73955, 144462,  32278,    997,  84486, 208976, 167332, 177775,  70679,
         113273, 119892,  37983,   2238,  22991, 135367,  11698,  41855,  31572,
         248075,      2,   -100,   -100]])

In [ ]:
!pip install evaluate -q
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import EvalPrediction

# Load the BLEU metric
metric = load_metric("sacrebleu")

def compute_metrics(eval_preds: EvalPrediction):
    preds, labels = eval_preds

    # If preds is a tuple, extract the first element
    if isinstance(preds, tuple):
        preds = preds[0]

    # Ensure that preds is a numpy array
    preds = np.array(preds)

    # If preds are logits, get the token IDs by taking argmax
    if preds.ndim == 3:  # Assuming shape (batch_size, sequence_length, vocab_size)
        preds = np.argmax(preds, axis=-1)

    # Replace -100 in labels with the tokenizer pad token ID
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process the decoded outputs
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # SacreBLEU metric expects references to be a list of lists
    decoded_labels = [[label] for label in decoded_labels]

    # Compute the BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Return the BLEU score
    return {"bleu": result["score"]}


<ipython-input-18-eec72d8ad47f>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


The repository for sacrebleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sacrebleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    model_ckpt,
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
from datasets import Dataset

validation_data=Dataset.from_dict(tokenized_inputs["validation"][:5])
validation_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_inputs["train"],
    eval_dataset=validation_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,1.217900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

TrainOutput(global_step=678, training_loss=1.1858401678304757, metrics={'train_runtime': 867.4207, 'train_samples_per_second': 6.253, 'train_steps_per_second': 0.782, 'total_flos': 543096729894912.0, 'train_loss': 1.1858401678304757, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.9780153036117554,
 'eval_bleu': 14.063042683868636,
 'eval_runtime': 0.659,
 'eval_samples_per_second': 7.587,
 'eval_steps_per_second': 1.517,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Model Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


events.out.tfevents.1720086505.612a524db36c.178.1:   0%|          | 0.00/407 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Govardhan-06/nllb-200-distilled-600M/commit/c64d882f832a6e9d27b90c928d7bea3e27a71fb2', commit_message='Model Training complete', commit_description='', oid='c64d882f832a6e9d27b90c928d7bea3e27a71fb2', pr_url=None, pr_revision=None, pr_num=None)